In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os
import csv
from collections import OrderedDict
import math
import datetime 
import warnings

warnings.filterwarnings("ignore") # use this if you want to get rid of unwanted warnings

In [ ]:
df = pd.read_csv('Time_series_dataset.csv')


In [ ]:
# if you need to work with data with specific interval, I would suggest using rank for simplicity
df ['rank_t'] = df.groupby('ID')['t'].rank() 

# if you need to update your target values with the mean for V-mask use below
df ['target_minus_updatedMean']= df['target'] - df['UpdatedMean']
df['CUMEAN']=df.groupby('ID')['target_minus_updatedMean'].cumsum(skipna = True)


# for v_mask codition Tr and Vr should be defined based on uper/lower arms and the coefficients
#please refer to (https://sixsigmastudyguide.com/cumulative-sum-chart-cusum/)
df['Tr'] = df_merged ['target_minus_updatedMean'] - (df_merged ['rank_t']*0.5)
df['Vr'] = df_merged ['target_minus_updatedMean'] + (df_merged ['rank_t']*0.5)


df_Max_Vr= df.set_index('t').groupby(['ID'])['Vr_EL_SV'].rolling('2D').max().reset_index().rename(columns={'Vr':'Max_Vr'})

df_Min_Tr= df.set_index('t').groupby(['ID'])['Tr_EL_SV'].rolling('2D').min().reset_index().rename(columns={'Tr':'Min_Tr'})


df_merged_Min_Max = pd.merge(df_Max_Vr, df_Min_Tr,  on=['ID','t'])
df_merged_Min_Max = pd.merge(df_merged_Min_Max, df,  on=['ID','t'])


In [ ]:
# applying upper/lower V_mask condition
df['Masked_CUMEAN'] = df_merged_Min_Max.apply(lambda row: row['CUMEAN'] if ((row['Max_Vr'] - row['Vr_']) <= 4  and (row['Tr'] - row['Min_Tr']) <= (4*row['Updated_std'])) 
    else "out of control", axis=1)
# note : row['Updated_std'] can be measured same as updated_mean